In [ ]:
!pip3 install pymupdf langchain openai python-dotenv langchain-community bs4 langchain-openai faiss-gpu

In [4]:
#from langchain.llms import OpenAI
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

from langchain_openai import OpenAIEmbeddings #criar os embbedings
from langchain_community.vectorstores import FAISS # vector DB FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter #Quebra o texto em pequenos chuncks


from langchain_core.prompts import ChatPromptTemplate #outra estratégia para criar prompts
from langchain.chains.combine_documents import create_stuff_documents_chain #junta o LLM com o prompt
from langchain.chains import create_retrieval_chain

import fitz


load_dotenv() #Pega tudo que ta no arquivo .env e cria as variaveis de ambiente, no nosso caso tem a key do openAI


True

In [2]:
path_pdf = '/dados/GoogleDrive/rotinas_python/RAG_LEARN/data/QA_RDC53.pdf'

pdf = fitz.open(path_pdf)

In [3]:
#pego o Texto completo do pdf

full_txt = ''.join([pdf[page_num].get_text() for page_num in range(pdf.page_count)][3:]).strip()
print(full_txt)

3 
 
 
GERÊNCIA GERAL DE MEDICAMENTOS E PRODUTOS BIOLÓGICOS 
 
Assunto: RDC 53/2015 e Guia 4/2015 
Edição 2.1 de 04/10/2017 
2. INTRODUÇÃO  
Este documento foi elaborado pela equipe da GGMED com a finalidade de prestar 
esclarecimentos quanto às perguntas mais frequentes relacionadas à RDC 53/2015 e 
ao Guia 4/2015. 
3. ESCOPO 
Este documento deve ser entendido no âmbito da GGMED, mais especificamente no 
âmbito dos medicamentos que estão sujeitos à RDC 53/2015, a menos que outro 
escopo esteja claramente descrito no contexto da pergunta. 
4. PERGUNTAS E RESPOSTAS 
4.1. ESTUDO 
DO 
PERFIL 
DE 
DEGRADAÇÃO 
E 
ESTUDO 
DE 
DEGRADAÇÃO FORÇADA 
4.1.1. Qual a diferença entre estudo do perfil de degradação e estudo de 
degradação forçada? 
O estudo do perfil de degradação é a análise crítica do perfil de degradação do 
medicamento, conforme disposto no Art. 6°, § 3º. Conforme Guia 4/2015:  
Estudo do Perfil de Degradação: estudo composto de parte crítica e, quando necessário, 
parte experimen

In [5]:
from langchain.schema import Document
def text_to_retriever(text: str):
    # Coloca o texto em uma lista de objetos Document
    docs = [Document(page_content=text)] 
    embeddings = OpenAIEmbeddings() # Cria embeddings
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100) # Divide o texto em partes menores
    
    documents = text_splitter.split_documents(docs) # Faz o split do texto
    print(len(documents),documents)
    vector = FAISS.from_documents(documents, embeddings) # Cria o vetor a partir dos documentos
    
    retriever = vector.as_retriever() # Converte o vetor em um retriever

    return retriever

In [6]:
res = text_to_retriever(full_txt)


86 [Document(page_content='3 \n \n \nGERÊNCIA GERAL DE MEDICAMENTOS E PRODUTOS BIOLÓGICOS \n \nAssunto: RDC 53/2015 e Guia 4/2015 \nEdição 2.1 de 04/10/2017 \n2. INTRODUÇÃO  \nEste documento foi elaborado pela equipe da GGMED com a finalidade de prestar \nesclarecimentos quanto às perguntas mais frequentes relacionadas à RDC 53/2015 e \nao Guia 4/2015. \n3. ESCOPO \nEste documento deve ser entendido no âmbito da GGMED, mais especificamente no \nâmbito dos medicamentos que estão sujeitos à RDC 53/2015, a menos que outro \nescopo esteja claramente descrito no contexto da pergunta. \n4. PERGUNTAS E RESPOSTAS \n4.1. ESTUDO \nDO \nPERFIL \nDE \nDEGRADAÇÃO \nE \nESTUDO \nDE \nDEGRADAÇÃO FORÇADA \n4.1.1. Qual a diferença entre estudo do perfil de degradação e estudo de \ndegradação forçada? \nO estudo do perfil de degradação é a análise crítica do perfil de degradação do \nmedicamento, conforme disposto no Art. 6°, § 3º. Conforme Guia 4/2015:  \nEstudo do Perfil de Degradação: estudo composto

In [7]:
llm = OpenAI(temperature=0.5, model='gpt-3.5-turbo-instruct')
prompt = ChatPromptTemplate.from_template(""" Responda a pergunta com base apenas no contexto
{context}
Pergunta: {input}
""")

doc_chain = create_stuff_documents_chain(llm, prompt) # Junta a LLM com o prompt

retriever = text_to_retriever(full_txt) # Usa o texto do PDF como base para o retriever

retriever_chain = create_retrieval_chain(retriever, doc_chain) # Liga o retriever com a chain LLM

input_prompt = {"input": "Como estabeleço um limite para uma impureza em um fármaco?"}
response = retriever_chain.invoke(input_prompt)

/tmp/ipykernel_79866/3098698593.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(temperature=0.5, model='gpt-3.5-turbo-instruct')


86 [Document(page_content='3 \n \n \nGERÊNCIA GERAL DE MEDICAMENTOS E PRODUTOS BIOLÓGICOS \n \nAssunto: RDC 53/2015 e Guia 4/2015 \nEdição 2.1 de 04/10/2017 \n2. INTRODUÇÃO  \nEste documento foi elaborado pela equipe da GGMED com a finalidade de prestar \nesclarecimentos quanto às perguntas mais frequentes relacionadas à RDC 53/2015 e \nao Guia 4/2015. \n3. ESCOPO \nEste documento deve ser entendido no âmbito da GGMED, mais especificamente no \nâmbito dos medicamentos que estão sujeitos à RDC 53/2015, a menos que outro \nescopo esteja claramente descrito no contexto da pergunta. \n4. PERGUNTAS E RESPOSTAS \n4.1. ESTUDO \nDO \nPERFIL \nDE \nDEGRADAÇÃO \nE \nESTUDO \nDE \nDEGRADAÇÃO FORÇADA \n4.1.1. Qual a diferença entre estudo do perfil de degradação e estudo de \ndegradação forçada? \nO estudo do perfil de degradação é a análise crítica do perfil de degradação do \nmedicamento, conforme disposto no Art. 6°, § 3º. Conforme Guia 4/2015:  \nEstudo do Perfil de Degradação: estudo composto

In [8]:
response

{'input': 'Como estabeleço um limite para uma impureza em um fármaco?',
 'context': [Document(page_content='meramente analítica. \n4.5.33. \nComo devem ser estabelecidos os limites de produtos de \ndegradação em medicamentos específicos e em medicamentos de baixo \nrisco sujeitos a notificação simplificada? \nQuando a monografia farmacopeica se referir ao produto acabado, o limite a ser \nestabelecido deve ser o fixado na monografia farmacopeica. Se houver limites diferentes \nfixados em monografias diferentes, o valor mais crítico deve ser adotado. Nos casos em \nque o produto de degradação estiver listado na monografia do IFA, a especificação para \no produto acabado deve ser estabelecida pela empresa e o racional para definição da \nespecificação deve ser apresentado, considerando-se a quantidade de IFA adicionada \nao medicamento. Nos casos de produtos de degradação tóxicos identificados na \nliteratura, as especificações devem ser definidas com base nos dados de toxicidade \ndispo

In [9]:
print(response['answer'])


Resposta: O limite para uma impureza em um fármaco deve ser estabelecido com base na monografia farmacopeica do produto acabado, considerando-se o valor mais crítico entre as diferentes monografias, caso haja diferenças. Nos casos em que o produto de degradação não estiver listado na monografia do insumo farmacêutico ativo (IFA), a empresa deve estabelecer a especificação para o produto acabado e apresentar o racional para essa definição, levando em consideração a quantidade de IFA adicionada ao medicamento. Em casos de produtos de degradação tóxicos descritos na literatura, as especificações devem ser definidas com base nos dados de toxicidade disponíveis para a substância monitorada. Em situações em que não houver estudos de estabilidade disponíveis, é necessário comprovar que a impureza em questão é uma impureza de síntese e não é formada durante o armazenamento do produto, não podendo ser considerada um produto de degradação.
